In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate
from utils_python import *

#plotting
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(420)
pd.set_option('display.max_columns', None)  # To show all columns

In [2]:
# Read dataset with encoding
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/filtered_dataset_with_geo_info_with_encoding.parquet.gzip")

In [3]:
df.head()

yearOfLoss  elevatedBuildingIndicator  totalBuildingInsuranceCoverage  \
0        1996                          0                        100000.0   
1        2009                          0                        150000.0   
2        2004                          0                        239300.0   
3        2004                          0                        102500.0   
4        1996                          0                         73700.0   

   totalContentsInsuranceCoverage  primaryResidenceIndicator  \
0                         25000.0                          1   
1                             0.0                          0   
2                         45300.0                          1   
3                         44800.0                          1   
4                          4300.0                          0   

   buildingPropertyValue  buildingReplacementCost  waterDepth  \
0               100000.0                      0.0         0.0   
1                37260.0                 113263.0         1.0   
2               191440.0                 239300.0         0.0   
3                82000.0                 102500.0         0.0   
4                73700.0                      0.0         0.0   

   rentalPropertyIndicator  postFIRMConstructionIndicator  \
0                        0                              1   
1                        0                              0   
2                        0                              0   
3                        0                              0   
4                        0                              1   

   buildingrelativeDamage  AgeofBuildinguntilflood  AgeofPolicyuntilflood  \
0                0.052520                 8.191650               0.164271   
1                0.083494                57.366188               1.273101   
2                0.011737                38.212183              28.440794   
3                0.800366                56.208077              17.459274   
4                0.070583                16.810404               9.180014   

      svd_1     svd_2     svd_3     svd_4     svd_5     svd_6     svd_7  \
0 -0.191515 -0.277861  0.438251 -0.135808  0.257442 -0.905254  0.517394   
1 -0.440167 -0.026508 -0.265916 -0.101934 -0.578929 -0.273033 -0.381519   
2 -0.487011 -0.178706  0.300662 -0.180206 -0.122099 -0.252817  0.428306   
3 -0.389182 -0.143250  0.341384 -0.081046  0.291763 -0.666212  1.571670   
4 -0.572386 -0.142794  0.306223  0.106274  0.035533 -0.221587  0.990321   

      svd_8     svd_9    svd_10    svd_11    svd_12    svd_13    svd_14  \
0  0.198713 -1.311131 -0.560012  0.519368  0.064180  0.600630  1.447684   
1 -0.066282 -0.761965 -0.364831 -0.789079  2.972900 -1.743709  4.187647   
2  0.024471 -0.140899 -0.029402 -0.572798 -0.472122  0.110230 -0.315755   
3 -0.164597  0.058252  0.231910  0.912889 -0.768089  0.094755  0.005752   
4 -0.207453  0.470000  0.379363  1.012849  0.526264  0.347955 -0.959901   

     svd_15    svd_16    svd_17    svd_18    svd_19    svd_20    svd_21  \
0  0.548300  1.195442 -1.206848 -1.677352 -0.290665 -0.008914  0.911473   
1 -1.028620  0.660021  0.302219  0.764221 -1.073185 -2.093600  0.419692   
2 -0.226288 -0.048457 -0.005177 -0.584027  1.562803 -0.141775  0.261002   
3  0.353880 -1.412354  1.136864  0.353490  1.320365 -0.010683 -0.476934   
4  0.987863  0.029553  0.721367  0.199539 -0.391917  0.175819 -2.181929   

     svd_22    svd_23    svd_24     svd_25    svd_26    svd_27    svd_28  \
0 -0.777211 -0.634360  0.905508   2.180291 -0.533245  0.392365  2.021945   
1  3.275569 -1.975996 -1.032234  22.982420 -3.494600 -1.374440  2.460046   
2 -0.439834 -0.024958  0.928115  -0.028791 -0.085646  0.008390  0.015467   
3 -0.085460  0.244494 -0.964397   0.431071 -0.234543 -0.055565 -0.255935   
4  0.966825 -0.223965 -0.775800  -0.592636  0.068290 -0.039304  0.535170   

     svd_29    svd_30  countyCode_00000  countyCode_01001  countyCode_01003  \
0 -0.417581 -0.243402                 0            

### Drop all county and yearOfLoss columns

In [4]:
# Identify columns that start with "countyCode"
columns_to_remove = [col for col in df.columns if col.startswith("countyCode_")]

# Combine these columns "yearOfLoss" for removal
columns_to_drop = columns_to_remove + ["yearOfLoss"]

df = drop_columns(df, columns_to_drop)

### Scale the numeric columns and get sample df if needed 

In [5]:
# Scale if needed
df = scale_variables(df, "buildingrelativeDamage", scale=False)

# Sample if needed
df = sample_dataframe(df, 5000, sampling_required=True)

### Data creation for modeling

In [6]:
# Create a list of the SVD column names
svd_columns = [f'svd_{i}' for i in range(1, 31)]

# Create a new DataFrame with only the SVD columns
svd_df = df[svd_columns].copy()

# Drop the SVD columns from the original DataFrame
df.drop(columns=svd_columns, inplace=True)

In [7]:
# Create baseline dataframes
Y = df['buildingrelativeDamage'].values
X = df.drop(columns='buildingrelativeDamage').values

# Initialize a dictionary to store the MAE for baseline and each SVD variable inclusion
# Setting the baseline MAE to 0 and each SVD variable's MAE to 0 initially
mae_scores = {'Baseline': 0}
mae_scores.update({col: 0 for col in svd_columns})

### Baseline Model

In [8]:
# Define the base estimator with a criterion of absolute error
base_estimator = DecisionTreeRegressor(criterion='absolute_error', random_state=420)

# Perform 5 fold cross-validation
regr = AdaBoostRegressor(estimator=base_estimator, n_estimators=100, learning_rate=0.1, loss='linear', random_state=420)
cv_results = cross_validate(regr, X, Y, cv=5, scoring='neg_mean_absolute_error')
mae_scores['Baseline'] = -np.mean(cv_results['test_score'])

print(f"Baseline MAE: {mae_scores['Baseline']}")

Baseline MAE: 0.18137455866160218


### Check geoinfo column inclusion

In [ ]:
# Initialize an array to store the MAE for each selection round
svd_selection_cv_scores = np.zeros(30)

# Variable selection and iterative cross-validation
for i in range(1, 31):
    # Identify the SVD variable with the lowest MAE
    best_svd_var, svd_selection_cv_scores[i-1] = svd_selection_ada(X, svd_df, Y)

    # Add the best_svd_var to X
    X = np.concatenate([X, svd_df[[best_svd_var]].values], axis=1)
    
    # Drop the included svd column from the svd dataframe
    svd_df = svd_df.drop(columns=best_svd_var)
    
    # Rerun 5-fold CV with the newly included SVD variable
    regr = AdaBoostRegressor(estimator=base_estimator, n_estimators=100, learning_rate=0.1, loss='linear', random_state=420)
    cv_results = cross_validate(regr, X, Y, cv=5, scoring='neg_mean_absolute_error')
    mae_scores[best_svd_var] = -np.mean(cv_results['test_score'])
    print(f"Iteration: {i}, Best SVD Var: {best_svd_var}, MAE: {mae_scores[best_svd_var]}")

Evaluating: svd_1 with MAE: 0.18128661371919857
Evaluating: svd_2 with MAE: 0.18120715694849945
Evaluating: svd_3 with MAE: 0.1807346643597147
Evaluating: svd_4 with MAE: 0.1817403573286123
Evaluating: svd_5 with MAE: 0.17934572270535865
Evaluating: svd_6 with MAE: 0.17948327980694678
Evaluating: svd_7 with MAE: 0.18338837956027884
Evaluating: svd_8 with MAE: 0.18251900960859785
Evaluating: svd_9 with MAE: 0.18113388124336036
Evaluating: svd_10 with MAE: 0.18232438920812816
Evaluating: svd_11 with MAE: 0.1820134681015791
Evaluating: svd_12 with MAE: 0.1798415256430201
Evaluating: svd_13 with MAE: 0.18111095517855175
Evaluating: svd_14 with MAE: 0.18164609221693895
Evaluating: svd_15 with MAE: 0.18326801404923776
Evaluating: svd_16 with MAE: 0.18165576948253587
Evaluating: svd_17 with MAE: 0.18185804798974037
Evaluating: svd_18 with MAE: 0.18268608544059978
Evaluating: svd_19 with MAE: 0.1801893966584086
Evaluating: svd_20 with MAE: 0.18238142062846133
Evaluating: svd_21 with MAE: 0.181

### Plots

In [ ]:
# Convert mae_scores to a pandas DataFrame
mae_df = pd.DataFrame(list(mae_scores.items()), columns=['Variable', 'MAE'])

# Save the DataFrame to a CSV file
mae_df.to_csv('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/adaboost_regression_mae_df.csv', index=False)

# Plotting
plt.figure(figsize=(10, 6))
sns.lineplot(data=mae_df, x='Variable', y='MAE', marker='o', color='darkgreen', markersize=8)

plt.xticks(rotation=45, ha='right')
plt.title('MAE Across Models with SVD Variable Inclusion (AdaBoost Regressor)', fontsize=14)
plt.xlabel('SVD Variable Included', fontsize=12)
plt.ylabel('Mean Absolute Error (MAE)', fontsize=12)
sns.despine()  # Removes the top and right spines
plt.grid(True, which='major', linestyle='--', linewidth='0.5', color='gray')  # Adds a grid
plt.tight_layout()  # Adjusts subplots to fit into the figure area.

plt.show()